# week 6

## goals:
- meet with Dan & Ray to determine next steps
- specifically try to answer:
    - "you might want to just focus on MPAs that are beyond the continental shelves but we can talk about that."
    - Should I include things like Ross Sea (which probably have a terrestrial component as well)?
    - How do I use the post-2016 data? I'm in the google [permission?] group – can I make queries against an API?
    - 

- another sanity check - plot just the points that are inside MPAs (use the joined points table before aggregation)


## notes/learnings:



In [1]:
import numpy as np
import pandas
from datetime import datetime
import matplotlib.pyplot as plt
import os
import pywdpa
import geopandas
import contextily as ctx
from shapely import geometry
import pretty_html_table

pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', 20)

In [2]:
# reads the downloaded WPDA polygon files
protected_areas = pandas.concat([
    geopandas.read_file("data/WDPA_WDOECM_wdpa_shp/WDPA_WDOECM_wdpa_shp0/WDPA_WDOECM_wdpa_shp-polygons.shp"),
    geopandas.read_file("data/WDPA_WDOECM_wdpa_shp/WDPA_WDOECM_wdpa_shp1/WDPA_WDOECM_wdpa_shp-polygons.shp"),
    geopandas.read_file("data/WDPA_WDOECM_wdpa_shp/WDPA_WDOECM_wdpa_shp2/WDPA_WDOECM_wdpa_shp-polygons.shp")
])
# filters for marine only (may want to change this to 1 or 2 (2 is marine only, 1 is mixed, 0 is terrestrial))
mpas = protected_areas[protected_areas["MARINE"] == "2"]
# delete the larger table - it's like 3 GiB and not needed past this point
del(protected_areas)

KeyboardInterrupt: 

In [ ]:
## Load the fishing hours data (this is kinda slow)
filenames = os.listdir('data/daily_csvs')

# this might be faster but the status printout is nice:
# points = pandas.concat([geopandas.read_file('data/daily_csvs/' + filename) for filename in filenames])

counted = 0
points = []
for filename in filenames:
    print(f'\r {filename} {counted}/{len(filenames)}', end='')
    points.append(pandas.read_csv('data/daily_csvs/' + filename,
                                      dtype={'lat_bin': 'int16',
                                             'lon_bin': 'int16',
                                             'mmsi': 'int32',
                                             'fishing_hours': 'float32'},
                                 parse_dates=['date']))
    counted += 1
points = pandas.concat(points) # deliberately overwriting points

In [ ]:
def compute_box(row, width=0.1):
    x = float(row['lon_bin'] * width)
    y = float(row['lat_bin'] * width)
    return geometry.box(x, y, x+width, y+width)

## Aggregate the points and bin them
points_aggregate = points.groupby(['lat_bin', 'lon_bin']).aggregate({'lat_bin': 'first', 'lon_bin': 'first', 'fishing_hours': 'sum'})
points_aggregate['geometry'] = points_aggregate.apply(compute_box, axis=1)
geopoints = geopandas.GeoDataFrame(points_aggregate, geometry=points_aggregate['geometry']).set_crs(epsg=4326)


In [ ]:
# Produce ranked list of mpas by amount of effort inside their borders
# q - should we join by id or something first?

mpas_ = mpas[['WDPAID', 'NAME', 'geometry', 'STATUS_YR', 'REP_M_AREA', 'REP_AREA', 'NO_TAKE']]
joined_points = geopandas.sjoin(geopoints, mpas_, op='within')


In [ ]:
def string_join(strings):
    base = strings.iloc[0] if strings.any() else ''
    for s in strings[1:]:
        base = base if s in base else base + '|' + s
    return base


joined_points_aggregate = joined_points.groupby(['WDPAID'], as_index=False).aggregate({
    'NAME': string_join,
    'fishing_hours': 'sum',
    'STATUS_YR': 'first',
    'REP_M_AREA': 'sum',
    'REP_AREA': 'sum',
    'NO_TAKE': string_join,
})



In [ ]:
sorted_mpas = joined_points_aggregate.sort_values("fishing_hours", ascending=False)

In [ ]:
since_2016 = sorted_mpas[sorted_mpas['STATUS_YR'] >= 2016]
since_2014 = sorted_mpas[sorted_mpas['STATUS_YR'] >= 2014]

f = open("2016_or_later.html", "w")
f.write(pretty_html_table.build_table(since_2016, 'blue_light'))
f.close()

f = open("2014_or_later.html", "w")
f.write(pretty_html_table.build_table(since_2014, 'blue_light'))
f.close()

f = open("all_mpas_by_fishing_hours.html", "w")
f.write(pretty_html_table.build_table(sorted_mpas, 'blue_light'))
f.close()

In [ ]:
## Plot it all!
ax = geopoints.plot(column='fishing_hours', figsize=(20, 30), markersize=1, cmap='Blues', scheme='quantiles', legend=True)

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

ax = world.plot(
    color='white', edgecolor='black', ax=ax, )



marae = mpas[mpas["WDPAID"] == 555624907.0]
pmnm = mpas[mpas["WDPAID"] == 220201.0]

LINEWIDTH = 0.5
mpas.plot(ax=ax, color='white', edgecolor='red', linewidth=LINEWIDTH)

plt.title('fishing hours 2012-2016 in 0.1 degree aggregate')
